In [1]:
# import of necessary libaries
# MIKE IO 1D, needs Pandas and Numpy
import mikeio1d
from mikeio1d.res1d import Res1D, QueryDataNode, QueryDataReach,ResultData, mike1d_quantities, ResultData
import pandas as pd
import numpy as np

# connection to MIKE+ database:
import sqlite3

# file and folder manipulation for input and output:
import os

# find files faster
from fnmatch import fnmatch

----

In [2]:
#location of mikeio
print(mikeio1d.__file__)

c:\users\tht\appdata\local\programs\python\python38\lib\site-packages\mikeio1d\__init__.py


------

# Function definition

In [3]:
print('Function definition...')

Function definition...


-----

In [4]:
def get_qfull_data(res1d):
    network_datas = list(res1d.data.NetworkDatas)
    for data in network_datas:
        if data.Quantity.Id == "Discharge of full reach":
            return data    

def get_qfull(reach, qfull_data):
    if qfull_data is None:
        return None

    reach_data = qfull_data.GetReachData(reach.Name)

    if reach_data is not None:
        return reach_data.GlobalValue

    return None


# Gets InvertLevel and GroundLevel from a Res1DManhole object:
def get_node_levels(node):
    try:
        invert_level = node.BottomLevel
    except:
        invert_level = None

    try:
        ground_level = node.GroundLevel
    except:
        ground_level = None

    return invert_level, ground_level

def get_diameter(reach):
    try:
        grid_points = list(reach.GridPoints)
        h_point = grid_points[0]
        diameter = round(h_point.CrossSection.Diameter,2)
    except:
        diameter = None

    return diameter

def get_slope(reach):
    grid_points = list(reach.GridPoints)
    gp_first = grid_points[0]
    gp_last = grid_points[-1]
    length = reach.Length
    slope = ((gp_first.Z - gp_last.Z) / length)*100
    return abs(slope)

def get_data_item(reach, quantity_id):
    item = None

    for data_item in list(reach.DataItems):
        if data_item.Quantity.Id == quantity_id:
            item = data_item
            break

    return item


# Gets min and max value and times for a any model element (not only reaches); needs
#     DHI.Mike1D.ResultDataAccess.Res1DManhole object or similar
#     list of DateTime objects
#     string with quantity (default ist "Discharge")

def get_minmax_value_result_file(reach, times_list, quantity_id="Discharge"):
    item = get_data_item(reach, quantity_id)

    min_value, min_time = None, None
    max_value, max_time = None, None   
    
    try:
        time_data = item.TimeData
        for time_step_index in range(time_data.NumberOfTimeSteps):
            
            for element_index in range(time_data.NumberOfElements):
                value = time_data.GetValue(time_step_index, element_index)
                if min_value is None or value < min_value:
                    min_value = value 
                    min_time = times_list[time_step_index].ToString()

                if max_value is None or value > max_value:
                    max_value = value 
                    max_time =  times_list[time_step_index].ToString()
    
    except:
        time_data = None
    
    return min_value, min_time, max_value, max_time


def get_minmax_value(reach, simulation_start, quantity_id="Discharge"):    
    item_max = get_data_item(reach, quantity_id + "Max")
    item_max_time = get_data_item(reach, quantity_id + "MaxTime")
    item_min = get_data_item(reach, quantity_id + "Min")
    item_min_time = get_data_item(reach, quantity_id + "MinTime")

    items = [item_max, item_max_time, item_min, item_min_time]
    if None in items:
        return None

    min_value, min_time = None, None
    max_value, max_time = None, None   

    number_of_elements = item_min.TimeData.NumberOfElements
    time_step_index = 0

    for element_index in range(number_of_elements):
        value = item_min.TimeData.GetValue(time_step_index, element_index)
        time = item_min_time.TimeData.GetValue(time_step_index, element_index)
        
        if min_value is None or value < min_value:
            min_value = value 
            min_time = simulation_start.AddSeconds(time)

        value = item_max.TimeData.GetValue(time_step_index, element_index)
        time = item_max_time.TimeData.GetValue(time_step_index, element_index)

        if max_value is None or value > max_value:
            max_value = value 
            max_time = simulation_start.AddSeconds(time)

    return min_value, min_time.ToString(), max_value, max_time.ToString()

def max_WL_start_end (quantity):
    try:
        #np-array of waterlevels
        wl_start = res1d.get_reach_start_values(reach.Name, quantity)
        wl_end = res1d.get_reach_end_values(reach.Name, quantity)

        #max values
        wl_start_max = wl_start.max()
        wl_end_max = wl_end.max()

        #corresponding time
        time_start=str(res1d.time_index[np.argmax(wl_start)])
        time_end=str(res1d.time_index[np.argmax(wl_start)])
        
    except:
        wl_start,wl_end,wl_start_max,wl_end_max,time_start,time_end= None, None, None, None, None, None
    
    return wl_start_max, time_start, wl_end_max, time_end

def q_max_q_full_ratio_data (q_minmax_data,qfull):
    try:
        qmax_qfull_ratio= q_minmax_data[2]/qfull

    except:
        q_minmax_data[2] is None or qfull is None
        qmax_qfull_ratio=None
    
    return qmax_qfull_ratio

In [5]:
def get_reach_type(reach):
    try:
        # get the full identifier string:
        fullstring = str(reach)
        
        # might be:
        # 'Res1DReach: B4.1520l1-13 (0-235.000625581015)'
        # 'Res1DReach: Weir:B4.1480w1-14 (0-1)',
        # 'Res1DReach: Pump:B4.1510p1-16 (0-80.0006092712283)'

        # get the second part after the ':'
        structureReach = fullstring.split(':')[1].lstrip()

        # if the second part is Pump or Weir, use Pump or Weir:
        if structureReach in ['Pump','Weir']:
            reach_type = structureReach

        # in any other case this must be a normal Link (Pipe or Canal or River)
        else:
            reach_type = 'Link'
    
    except:
        reach_type = None

    return reach_type
            
    

        


-----------

# Find res1d and sqlite files

In [6]:
print('Searching res1d and sqlite files...')

Searching res1d and sqlite files...


In [7]:
cwd = os.getcwd()

cwd

'E:\\Dokumente\\unt\\MIKEDoku\\MIKE+\\MenueErgebnisse\\mikeio1d-ergebnisbericht\\Skript\\script'

In [8]:
myFiles = os.listdir(cwd)

In [9]:
# create a list of res1d-files
myRes1dFiles = [file for file in myFiles if fnmatch(file, '*.res1d')]

# pick the first res1d-file
oneRes1dFile = myRes1dFiles[0]

oneRes1dFile

'Tutor1base.Copy.res1d'

In [10]:
# create a list of sqlite-files
mySQLiteFiles = [file for file in myFiles if fnmatch(file, '*.sqlite')] 


# pick the first sqlite-file
oneSQLiteFile = mySQLiteFiles[0]

oneSQLiteFile

'mup21Tutor1.sqlite'

In [11]:
altres1d = Res1D(oneRes1dFile)

In [12]:
altres1d

<mikeio1d.Res1D>
Start time: 1953-05-16 18:00:00
End time: 1953-05-16 21:59:05
# Timesteps: 157
# Catchments: 0
# Nodes: 16
# Reaches: 17
# Globals: 2
0 - WaterLevel <m>
1 - Discharge <m^3/s>
2 - TimeStep <sec>
3 - NumTriesForTimeStep <m>

In [13]:
#df = altres1d.read()

# Create nodes and reaches lists from res1d

In [14]:
print('Creating nodes and links lists from res1d...')

Creating nodes and links lists from res1d...


In [15]:
# create a Res1d-object
res1d = Res1D(oneRes1dFile)

reaches = list(res1d.data.Reaches)
nodes = list(res1d.data.Nodes)

qfull_data = get_qfull_data(res1d)
times_list  = list(res1d.data.TimesList)

simulation_start = res1d.data.StartTime

In [16]:
nodes

In [17]:
myNode = nodes[-1]
myNode.ID

'Weir Outlet:B4.1510w1'

In [18]:
# The list "nodes" contains special nodes at the end of weirs without defined end. 
# Their ID looks like 'Weir Outlet:B4.1510w1'. We want to eliminate those elements
# from the list.

#for i, n in enumerate(nodes):
#    try:
#        n.ID.split(':')[-2] == 'Weir Outlet'
#        nodes.pop(i)
#    except IndexError:
#        pass

#nodes

# Prepare desired node results

In [19]:
print('Preparing desired node results...')

Preparing desired node results...


In [20]:
# initialize node lists

Node_ID = []
WLmin = []
WLmax = []
WLmaxmin = []

# call necessary informations from initially defined functions

for node in nodes:
    id = node.ID
    min_wl = get_minmax_value_result_file(node, times_list, quantity_id="WaterLevel")[0]
    max_wl = get_minmax_value_result_file(node, times_list, quantity_id="WaterLevel")[2]
    max_min_wl = max_wl / min_wl
    
    # fill initialized node lists    
    Node_ID.append(id)
    WLmin.append(min_wl)
    WLmax.append(max_wl)
    WLmaxmin.append(max_min_wl)
    

In [21]:
# create dictionary with lists
dict_res1d_nodes = {'Node_ID':Node_ID, 'WLmin':WLmin, 'WLmax':WLmax, 'WLmaxmin':WLmaxmin}

# create dataframe, transpose
df_res1dNode=pd.DataFrame(dict_res1d_nodes)

df_res1dNode =  df_res1dNode.set_index('Node_ID')

df_res1dNode


,WLmin,WLmax,WLmaxmin
Node_ID,,,
B4.1520,16.50250,19.50122,1.181713
B4.1500,16.70300,19.61063,1.174078
B4.1501,17.60150,20.76025,1.179459
B4.1502,17.86125,21.78367,1.219605
B4.1490,16.39500,17.43768,1.063597
B4.1485,16.35500,17.37703,1.062490
B4.1320,17.10500,17.51902,1.024205
B4.1310,17.30500,17.66968,1.021074
B4.1300,17.42500,17.88718,1.026524


# Prepare desired Link results

In [22]:
print('Preparing desired link results...')

Preparing desired link results...


In [23]:
#initialize lists
Link_ID=[]
Reach_Type=[]
From_Node_ID=[]
Invert_level_from_Node=[]
Ground_level_from_Node=[]
To_Node_ID=[]
Invert_level_to_Node=[]
Ground_level_to_Node=[]
Length=[]
Diameter=[]
Slope=[]
Qfull=[]

Vmin=[]
t_Vmin=[]
Vmax=[]
t_Vmax=[]
Qmin=[]
t_Qmin=[]
Qmax=[]
t_Qmax=[]

Qmax_Qfull=[]
WLmax_start=[]
WLmax_start_time=[]
WLmax_end=[]
WLmax_end_time=[]



for reach in reaches:
    #call necessary informations from initially defined functions
    name = reach.Name
    reachtype = get_reach_type(reach)
    node_from_index = reach.StartNodeIndex
    node_to_index = reach.EndNodeIndex

    node_from = nodes[node_from_index]
    node_to = nodes[node_to_index]

    invert_level_from, ground_level_from = get_node_levels(node_from)
    invert_level_to, ground_level_to = get_node_levels(node_to)

    diameter = get_diameter(reach)

    slope = get_slope(reach)

    qfull = get_qfull(reach, qfull_data)

    v_minmax_data = get_minmax_value_result_file(reach, times_list, "FlowVelocity")
    q_minmax_data = get_minmax_value_result_file(reach, times_list, "Discharge")
       
    maxWL=max_WL_start_end('WaterLevel')

    Qmax_Qfull_ratio = q_max_q_full_ratio_data(q_minmax_data,qfull)

    #fill initialized lists
    Link_ID.append(name)
    Reach_Type.append(reachtype)
    From_Node_ID.append(node_from.ID)
    Invert_level_from_Node.append(invert_level_from)
    Ground_level_from_Node.append(ground_level_from)
    To_Node_ID.append(node_to.ID)
    Invert_level_to_Node.append(invert_level_to)
    Ground_level_to_Node.append(ground_level_to)
    Diameter.append(diameter)
    Length.append(reach.Length)
    Slope.append(slope)
    Vmin.append(v_minmax_data[0])
    t_Vmin.append(v_minmax_data[1])
    Vmax.append(v_minmax_data[2])
    t_Vmax.append(v_minmax_data[3])
    Qmin.append(q_minmax_data[0])
    t_Qmin.append(q_minmax_data[1])
    Qmax.append(q_minmax_data[2])
    t_Qmax.append(q_minmax_data[3])
    Qfull.append(qfull)
    Qmax_Qfull.append(Qmax_Qfull_ratio)
    WLmax_start.append(maxWL[0])
    WLmax_start_time.append(maxWL[1])
    WLmax_end.append(maxWL[2])
    WLmax_end_time.append(maxWL[3])
    
#### Der Rest des Skripts war eingerückt, also Teil des for-loops. Aber das bedeutet
#### doch, dass der Dataframe nach jeder neuen Zeile erzeugt wird. Es reicht, wenn
#### man das am Ende macht.

#convert lists into list of lists and further into a dataframe    


# list_res1d_prop=[Link_ID,Reach_Type,From_Node_ID,Invert_level_from_Node,
#                    Ground_level_from_Node,To_Node_ID,
#                    Invert_level_to_Node,Ground_level_to_Node,
#                    Diameter,Length,Slope,Vmin,t_Vmin,Vmax,t_Vmax,Qmin,t_Qmin,Qmax,t_Qmax,Qfull,Qmax_Qfull,WLmax_start,WLmax_start_time,WLmax_end,WLmax_end_time]

dict_res1d_prop = {'Link_ID':Link_ID, 'Reach_Type':Reach_Type, 'From_Node_ID':From_Node_ID,'Invert_level_from_Node':Invert_level_from_Node, 
                   'Ground_level_from_Node':Ground_level_from_Node,'To_Node_ID':To_Node_ID, 'Invert_level_to_Node': Invert_level_to_Node,
                   'Ground_level_to_Node':Ground_level_to_Node, 'Diameter': Diameter,'Length':Length, 'Slope':Slope,'Vmin':Vmin, 
                   't_Vmin':t_Vmin,'V,ax':Vmax, 't_Vmax':t_Vmax, 'Qmin':Qmin, 't_Qmin':t_Qmin,'Qmax':Qmax, 't_Qmax':t_Qmax,
                   'Qfull':Qfull,'Qmax_Qfull':Qmax_Qfull, 'WLmax_start':WLmax_start, 'WLmax_start_time':WLmax_start_time, 'WLmax_end':WLmax_end,
                   'WLmax_end_time':WLmax_end_time}


df_res1dLink=pd.DataFrame(dict_res1d_prop)

# define column names of final dataframe    
#df_res1dLink=['Haltungstyp','Schacht oben','Sohlhöhe_Schacht oben',
                # 'Deckenhöhe_Schacht oben','Schacht unten','Sohlhöhe_Schacht unten',
                # 'Deckenhöhe_Schacht unten','Durchmesser [m]','Länge [m]','Gefälle [%]',
                # 'Vmin [m/s]','Vmin_Zeitpunkt','Vmax [m/s]','Vmax_Zeitpunkt','Qmin [m/s]',
                # 'Qmin_Zeitpunkt','Qmax [m/s]','Qmax_Zeitpunkt','Qvoll [m³/s]','Qmax/Qvoll [%]',
                # 'Wspmax Schacht oben','Wspmax Schacht oben Zeitpunkt','Wspmax Schacht unten','Wspmax Schacht Zeitpunkt']

In [24]:
#print filename
print(res1d.file_path)

#visualize first 5 rows of dataframe
df_res1dLink.head()

Tutor1base.Copy.res1d


,Link_ID,Reach_Type,From_Node_ID,Invert_level_from_Node,Ground_level_from_Node,To_Node_ID,Invert_level_to_Node,Ground_level_to_Node,Diameter,Length,...,Qmin,t_Qmin,Qmax,t_Qmax,Qfull,Qmax_Qfull,WLmax_start,WLmax_start_time,WLmax_end,WLmax_end_time
0,B4.1200l1,Link,B4.1200,16.799999,19.900000,B4.1485,16.35,19.799999,0.7,479.998999,...,-0.005379,16.05.1953 18:20:30,0.423308,16.05.1953 18:21:30,0.141865,2.983871,20.247484,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23
1,B4.1300l1,Link,B4.1300,17.420000,19.900000,B4.1320,17.10,20.200001,NaN,299.999956,...,0.000191,16.05.1953 21:57:16,0.386263,16.05.1953 18:26:37,-1.000000,-0.386263,17.843431,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37
2,B4.1310l1,Link,B4.1310,17.299999,20.230000,B4.1320,17.10,20.200001,NaN,323.110187,...,0.000146,16.05.1953 21:57:16,0.213843,16.05.1953 18:26:37,-1.000000,-0.213843,17.646738,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37
3,B4.1320l1,Link,B4.1320,17.100000,20.200001,B4.1485,16.35,19.799999,NaN,270.002056,...,-0.228450,16.05.1953 18:25:30,0.430228,16.05.1953 18:30:35,-1.000000,-0.430228,17.444399,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23
4,B4.1480l1,Link,B4.1480,16.250000,18.500000,A0.0327,16.00,17.500000,1.2,119.998586,...,-0.804957,16.05.1953 18:50:37,1.260161,16.05.1953 18:34:11,1.966383,0.640852,17.127731,1953-05-16 18:50:37,17.187971,1953-05-16 18:50:37


---

In [25]:
# Behalte nur die Zeilen, in denen Haltungstyp = 'Link' ist

#df_res1dLink = df_res1dLink.loc[df['Reach_Type'] == 'Link']

# set index on Link_ID:
df_res1dLink =  df_res1dLink.set_index('Link_ID')

df_res1dLink

,Reach_Type,From_Node_ID,Invert_level_from_Node,Ground_level_from_Node,To_Node_ID,Invert_level_to_Node,Ground_level_to_Node,Diameter,Length,Slope,...,Qmin,t_Qmin,Qmax,t_Qmax,Qfull,Qmax_Qfull,WLmax_start,WLmax_start_time,WLmax_end,WLmax_end_time
Link_ID,,,,,,,,,,,,,,,,,,,,,
B4.1200l1,Link,B4.1200,16.799999,19.900000,B4.1485,16.350000,19.799999,0.70,479.998999,0.093750,...,-5.378956e-03,16.05.1953 18:20:30,0.423308,16.05.1953 18:21:30,0.141865,2.983871,20.247484,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23
B4.1300l1,Link,B4.1300,17.420000,19.900000,B4.1320,17.100000,20.200001,NaN,299.999956,0.106667,...,1.912120e-04,16.05.1953 21:57:16,0.386263,16.05.1953 18:26:37,-1.000000,-0.386263,17.843431,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37
B4.1310l1,Link,B4.1310,17.299999,20.230000,B4.1320,17.100000,20.200001,NaN,323.110187,0.061898,...,1.455536e-04,16.05.1953 21:57:16,0.213843,16.05.1953 18:26:37,-1.000000,-0.213843,17.646738,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37
B4.1320l1,Link,B4.1320,17.100000,20.200001,B4.1485,16.350000,19.799999,NaN,270.002056,0.277776,...,-2.284498e-01,16.05.1953 18:25:30,0.430228,16.05.1953 18:30:35,-1.000000,-0.430228,17.444399,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23
B4.1480l1,Link,B4.1480,16.250000,18.500000,A0.0327,16.000000,17.500000,1.20,119.998586,0.208336,...,-8.049570e-01,16.05.1953 18:50:37,1.260161,16.05.1953 18:34:11,1.966383,0.640852,17.127731,1953-05-16 18:50:37,17.187971,1953-05-16 18:50:37
B4.1485l1,Link,B4.1485,16.350000,19.799999,B4.1480,16.250000,18.500000,1.20,60.000457,0.166666,...,-2.724102e-02,16.05.1953 18:18:30,1.448613,16.05.1953 18:34:11,1.758770,0.823651,17.135504,1953-05-16 18:51:47,17.127731,1953-05-16 18:51:47
B4.1490l1,Link,B4.1490,16.389999,19.799999,B4.1485,16.350000,19.799999,1.20,149.998814,0.026666,...,2.453081e-05,16.05.1953 18:01:06,0.834468,16.05.1953 18:26:37,0.703513,1.186145,17.403952,1953-05-16 18:37:23,17.377028,1953-05-16 18:37:23
B4.1491l1,Link,B4.1491,17.500000,20.150000,B4.1490,16.389999,19.799999,0.35,239.999500,0.041667,...,1.467786e-06,16.05.1953 18:01:06,0.152876,16.05.1953 18:29:05,0.030966,4.936929,20.108583,1953-05-16 18:30:35,17.687538,1953-05-16 18:30:35
B4.1500l1,Link,B4.1500,16.700001,19.200001,B4.1490,16.389999,19.799999,0.60,264.002011,0.117424,...,-1.992645e-02,16.05.1953 18:18:30,0.481612,16.05.1953 18:24:30,0.185997,2.589350,19.338707,1953-05-16 18:34:11,17.437677,1953-05-16 18:34:11


# Join Node results with msm_Node

In [26]:
print('Joining results with msm_Node information...')

Joining results with msm_Node information...


In [27]:
# establish connection to MIKE+ database:
con = sqlite3.connect(oneSQLiteFile)

In [28]:
# pick two columns from 'msm_Link':
df_msmNode = pd.read_sql_query("SELECT muid, description, assetname from msm_Node", con)

In [29]:
# set index on 'muid'
df_msmNode = df_msmNode.set_index('muid')

In [30]:
df_msmNode.head()

,description,assetname
muid,,
B4.1520,None,None
B4.1500,None,Nordknoten
B4.1501,None,None
B4.1502,None,Nordwestknoten
B4.1490,None,None


In [31]:
# join...werden dadurch die fälschlichen Knoten der Wehre entfernt?
df_res1dmsmNode = pd.merge(df_res1dNode, df_msmNode, left_index=True, right_index=True)

df_res1dmsmNode

,WLmin,WLmax,WLmaxmin,description,assetname
B4.1520,16.50250,19.50122,1.181713,None,None
B4.1500,16.70300,19.61063,1.174078,None,Nordknoten
B4.1501,17.60150,20.76025,1.179459,None,None
B4.1502,17.86125,21.78367,1.219605,None,Nordwestknoten
B4.1490,16.39500,17.43768,1.063597,None,None
B4.1485,16.35500,17.37703,1.062490,None,Kreuzknoten
B4.1320,17.10500,17.51902,1.024205,None,None
B4.1310,17.30500,17.66968,1.021074,None,Ostknoten
B4.1300,17.42500,17.88718,1.026524,None,None
B4.1200,16.80350,20.37670,1.212646,None,None


# Join Link results with msmLink

In [32]:
print('Joining Link results with msm_Link information...')

Joining Link results with msm_Link information...


In [33]:
# establish connection to MIKE+ database:
con = sqlite3.connect(oneSQLiteFile)

In [34]:
# pick two columns from 'msm_Link':
df_msmLink = pd.read_sql_query("SELECT muid, description, assetname from msm_Link", con)

In [35]:
# set index on 'muid'
df_msmLink = df_msmLink.set_index('muid')

In [36]:
df_msmLink.head()

,description,assetname
muid,,
B4.1200l1,None,None
B4.1300l1,None,None
B4.1310l1,None,None
B4.1320l1,None,None
B4.1480l1,None,None


In [37]:
df_res1dmsmLink = pd.merge(df_res1dLink, df_msmLink, left_index=True, right_index=True)

df_res1dmsmLink

,Reach_Type,From_Node_ID,Invert_level_from_Node,Ground_level_from_Node,To_Node_ID,Invert_level_to_Node,Ground_level_to_Node,Diameter,Length,Slope,...,Qmax,t_Qmax,Qfull,Qmax_Qfull,WLmax_start,WLmax_start_time,WLmax_end,WLmax_end_time,description,assetname
B4.1200l1,Link,B4.1200,16.799999,19.900000,B4.1485,16.350000,19.799999,0.70,479.998999,0.093750,...,0.423308,16.05.1953 18:21:30,0.141865,2.983871,20.247484,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23,None,None
B4.1300l1,Link,B4.1300,17.420000,19.900000,B4.1320,17.100000,20.200001,NaN,299.999956,0.106667,...,0.386263,16.05.1953 18:26:37,-1.000000,-0.386263,17.843431,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37,None,None
B4.1310l1,Link,B4.1310,17.299999,20.230000,B4.1320,17.100000,20.200001,NaN,323.110187,0.061898,...,0.213843,16.05.1953 18:26:37,-1.000000,-0.213843,17.646738,1953-05-16 18:26:37,17.519020,1953-05-16 18:26:37,None,None
B4.1320l1,Link,B4.1320,17.100000,20.200001,B4.1485,16.350000,19.799999,NaN,270.002056,0.277776,...,0.430228,16.05.1953 18:30:35,-1.000000,-0.430228,17.444399,1953-05-16 18:32:23,17.377028,1953-05-16 18:32:23,None,None
B4.1480l1,Link,B4.1480,16.250000,18.500000,A0.0327,16.000000,17.500000,1.20,119.998586,0.208336,...,1.260161,16.05.1953 18:34:11,1.966383,0.640852,17.127731,1953-05-16 18:50:37,17.187971,1953-05-16 18:50:37,None,None
B4.1485l1,Link,B4.1485,16.350000,19.799999,B4.1480,16.250000,18.500000,1.20,60.000457,0.166666,...,1.448613,16.05.1953 18:34:11,1.758770,0.823651,17.135504,1953-05-16 18:51:47,17.127731,1953-05-16 18:51:47,None,None
B4.1490l1,Link,B4.1490,16.389999,19.799999,B4.1485,16.350000,19.799999,1.20,149.998814,0.026666,...,0.834468,16.05.1953 18:26:37,0.703513,1.186145,17.403952,1953-05-16 18:37:23,17.377028,1953-05-16 18:37:23,None,None
B4.1491l1,Link,B4.1491,17.500000,20.150000,B4.1490,16.389999,19.799999,0.35,239.999500,0.041667,...,0.152876,16.05.1953 18:29:05,0.030966,4.936929,20.108583,1953-05-16 18:30:35,17.687538,1953-05-16 18:30:35,None,None
B4.1500l1,Link,B4.1500,16.700001,19.200001,B4.1490,16.389999,19.799999,0.60,264.002011,0.117424,...,0.481612,16.05.1953 18:24:30,0.185997,2.589350,19.338707,1953-05-16 18:34:11,17.437677,1953-05-16 18:34:11,None,None
B4.1501l1,Link,B4.1501,17.600000,20.469999,B4.1500,16.700001,19.200001,0.30,301.496248,0.033168,...,0.093888,16.05.1953 18:22:30,0.018316,5.126104,20.711884,1953-05-16 18:32:23,19.610634,1953-05-16 18:32:23,None,None


# define output res1d-report format (*.csv | *.xlsx)

In [38]:
print('Preparing your output...')

Preparing your output...


---

In [39]:
# extract the root of the filename even if filename contains a dot, hence better than split('.')
rootName = os.path.splitext(oneRes1dFile)[0]

# export NODE result table to csv
df_res1dmsmNode.to_csv(rootName + '_Knoten.csv', index_label='MUID',header=['Min.WSP','Max.WSP','Max/Min.WSP','AssetName','Beschreibung'])

print(rootName + '_Knoten.csv exportiert')

# export LINK result table to csv
# root name .... columns= engl. column names s. oben in reihenfolge excel und dann header umstellen 
df_res1dmsmLink.to_csv(rootName + '_Haltungen.csv', index_label='MUID', header=['Haltungstyp','Schacht oben','Sohlhöhe_Schacht oben',
                'Deckenhöhe_Schacht oben','Schacht unten','Sohlhöhe_Schacht unten',
                'Deckenhöhe_Schacht unten','Durchmesser [m]','Länge [m]','Gefälle [%]',
                'Vmin [m/s]','Vmin_Zeitpunkt','Vmax [m/s]','Vmax_Zeitpunkt','Qmin [m/s]',
                'Qmin_Zeitpunkt','Qmax [m/s]','Qmax_Zeitpunkt','Qvoll [m³/s]','Qmax/Qvoll [%]',
                'Wspmax Schacht oben','Wspmax Schacht oben Zeitpunkt','Wspmax Schacht unten','Wspmax Schacht Zeitpunkt','Beschreibung', 'AssetName'])

print(rootName + '_Haltungen.csv exportiert')

#export result table to csv without index, a specified sheeetname & 2 decimal places
#df.to_excel(res1d.file_path.split('.')[0]+'.xlsx',index=False, sheet_name='res1d_network_properties',float_format = "%0.3f")



Tutor1base.Copy_Knoten.csv exportiert
Tutor1base.Copy_Haltungen.csv exportiert


In [40]:
# os.system("pause")

In [41]:
# Eingabe abwarten, um Kosole zu schließen
input('Zum Beenden ENTER druecken')

Zum Beenden ENTER druecken 


''